# Тест на нормальное распределение

In [1]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
fig_width = 6.3  
fig_height = 3.54  
plt.rcParams['figure.figsize'] = (fig_width, fig_height) # установка размеров для всех графиков

Фильтрация данных


In [3]:
file_path = "D:/AI/Computerized-campimetry-test-analysis/summary_per_color_group.xlsx"
data = pd.read_excel(file_path)
necessary = data.rename(columns=lambda x: x.replace('T258', 'CM')) #замена названий столбцов

In [4]:
test_data = necessary[["CM_dH+_yellow_green_50%", "CM_dH+_blue_50%","CM_dH+_purple_50%","CM_dH+_red_50%","id_test_attempt"]]
test_data.to_csv("data_for_analysis.csv", index=False) #сохранение для другого ноутбука
test_data = test_data[test_data["id_test_attempt"]==0]
test_data = test_data.drop("id_test_attempt", axis=1) #удаление колонки. axis=1 колонки, axis=0 строки
test_data.to_csv("data_for_scatterplots.csv", index=False)

test_data

,CM_dH+_yellow_green_50%,CM_dH+_blue_50%,CM_dH+_purple_50%,CM_dH+_red_50%
0,4.0,1.0,2.0,3.0
1,4.0,1.0,2.0,2.5
2,2.0,1.0,1.0,1.5
3,4.0,1.0,3.0,2.0
6,5.0,1.0,3.0,3.0
...,...,...,...,...
2998,4.0,3.0,6.0,7.5
2999,9.0,4.0,6.0,7.5
3000,5.0,3.0,9.0,10.0
3001,28.0,8.0,9.0,13.0


## Построение графиков qq plot

Графики, которые помогают визуально оценить, насколько данные соответствуют нормальному распределению.Если данные соответствуют нормальному распределению, точки лягут вдоль прямой.

In [5]:
def build_qq_plots(data):
    for column in test_data.columns:
        plt.figure(figsize=(fig_width,fig_height), dpi=300)
        stats.probplot(data[column], dist="norm", plot=plt) 
        plt.title(f"Qq plot для {column}", fontsize=16, fontweight='light')
        plt.xlabel("Теоретические квантили", fontsize=14, fontweight='light')
        plt.ylabel("Упорядоченные значения", fontsize=14, fontweight='light')

## Тест Шапиро-Уилка

Статистический тест, который проверяет, насколько данные соответствуют нормальному распределению. Нулевая гипотеза (H0): данные имеют нормальное распределение. Альтернативная гипотеза (H1): данные не имеют нормального распределения.

In [6]:
def shapiro_wilk_test(data):
    columns = ["Столбец", "p-value", "Результат"]
    result_table = pd.DataFrame(columns = columns)
    for i, column in enumerate(data.columns):
        _, p_val = stats.shapiro(data[column]) # пропускается stat, где хранятся числа от 0 до 1(0- близко к нормальному, 1- сильное отклонение)
        if p_val < 0.05:
            status = "Гипотеза отклонена. Распределение не нормальное"
        else:
            status = "Гипотеза подтверждена. Распределение нормальное"
        result_table.loc[i] = [column, p_val, status]
    # Настройка ширины таблицы и вывод на экран
    pd.set_option('display.width', 130)
    print(result_table)            
    

In [7]:
shapiro_wilk_test(test_data)

                   Столбец  p-value                                        Результат
0  CM_dH+_yellow_green_50%      0.0  Гипотеза отклонена. Распределение не нормальное
1          CM_dH+_blue_50%      0.0  Гипотеза отклонена. Распределение не нормальное
2        CM_dH+_purple_50%      0.0  Гипотеза отклонена. Распределение не нормальное
3           CM_dH+_red_50%      0.0  Гипотеза отклонена. Распределение не нормальное


## Тест Краскала Уоллиса

Тест Краскала-Уоллиса — тест для проверки статистических различий между тремя и более независимыми группами. Он является аналогом ANOVA для данных, которые не следуют нормальному распределению. Нулевая гипотеза (H0): все группы имеют одинаковое распределение. Альтернативная гипотеза (H1): хотя бы одна из групп имеет другое распределение.

In [1]:
def kruskal_wallis_test(data):
    groups = list()
    for col in data.columns:
        groups.append(data[col])

    _, p_val = stats.kruskal(*groups)

    print(f"p-value = {p_val}")

    if p_val < 0.05:
        print("Гипотеза отклонена. Есть статистические различия между группами")
    else:
        print("Гипотеза подтверждена. Все группы имеют одинаковое распределение")

In [9]:
kruskal_wallis_test_for_4_groups(test_data)

p-value = 0.0
Гипотеза отклонена. Есть статистические различия между группами


### Пост-хок анализ


Пост-хок анализ включает в себя попарные тесты Манна-Уитни с коррекцией Бонферрони на множественные сравнения, чтобы уменьшить вероятность ошибки первого рода. Тест Манна-Уитни — тест, который используется для сравнения двух независимых выборок, когда предположение о нормальности данных нарушается. Он проверяет, различаются ли медианы двух групп, или, более точно, можно ли сказать, что одна группа имеет систематически большие значения, чем другая. Коррекция Бонферрони — это метод, используемый для уменьшения вероятности ошибки первого рода (то есть, ложного положительного результата) при множественных тестах. 
Когда проводятся несколько статистических тестов одновременно, вероятность того, что хотя бы один из них покажет ложноположительный результат, увеличивается. Чтобы компенсировать увеличение вероятности ложных срабатываний при множественных тестах, уменьшается пороговое значение (alpha) для каждого теста, деля его на количество проведенных тестов. На практике в силу того, что в статистических пакетах мы работаем с p-value, корректируется именно его значение (умножается p-value каждого теста на количество тестов). Таким образом, мы просто сравниваем уже скорретированное p-value.

In [10]:
def post_hoc_analysis(data):
    col = data.columns #названия
    pairs = list(combinations(col, 2)) #генерирует все возможные пары из списка столбцов. список кортежей получится

    post_hoc_tab = []

    for pair in pairs:
        a = data[pair[0]]
        b = data[pair[1]]
        _,p_val = stats.mannwhitneyu(a,b)   #тест Манна Уитни для 2 групп

        post_hoc_tab.append({        #список словарей
                'Группа 1': pair[0],
                'Группа 2': pair[1],
                'p-value': p_val
            })

    post_hoc_df = pd.DataFrame(post_hoc_tab)
    post_hoc_df["p-value после коррекции"] = post_hoc_df["p-value"] * len(pairs) #коррекция Бонферрони (умножение на количество экспериментов)

    post_hoc_df['Результат'] = post_hoc_df['p-value после коррекции'].apply(
        lambda x: "Гипотеза отклонена. Различия есть" if x < 0.05 else "Гипотеза подтверждена. Различий нет")

    print(post_hoc_df)

In [11]:
post_hoc_analysis(test_data)

                  Группа 1           Группа 2        p-value  p-value после коррекции                          Результат
0  CM_dH+_yellow_green_50%    CM_dH+_blue_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
1  CM_dH+_yellow_green_50%  CM_dH+_purple_50%  3.903005e-136            2.341803e-135  Гипотеза отклонена. Различия есть
2  CM_dH+_yellow_green_50%     CM_dH+_red_50%   1.738557e-77             1.043134e-76  Гипотеза отклонена. Различия есть
3          CM_dH+_blue_50%  CM_dH+_purple_50%  1.210989e-231            7.265931e-231  Гипотеза отклонена. Различия есть
4          CM_dH+_blue_50%     CM_dH+_red_50%  2.476211e-297            1.485727e-296  Гипотеза отклонена. Различия есть
5        CM_dH+_purple_50%     CM_dH+_red_50%   1.344621e-10             8.067725e-10  Гипотеза отклонена. Различия есть


Функция для реализации тестов

In [12]:
def stat_test (test, data, stat_tests_dict):
    if test in stat_tests_dict:
        stat_tests_dict[test](data)
    else:
        raise ValueError(f"Тест не найден. Доступные тесты: {stat_test_dict.keys()}")
    
   

In [13]:
 stat_tests_dict = {
                "post_hoc_analysis" : post_hoc_analysis,
                "kruskal_wallis_test_for_4_groups" : kruskal_wallis_test_for_4_groups,
                "shapiro_wilk_test" : shapiro_wilk_test
                 }
stat_test("post_hoc_analysis", test_data, stat_tests_dict)
     

                  Группа 1           Группа 2        p-value  p-value после коррекции                          Результат
0  CM_dH+_yellow_green_50%    CM_dH+_blue_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
1  CM_dH+_yellow_green_50%  CM_dH+_purple_50%  3.903005e-136            2.341803e-135  Гипотеза отклонена. Различия есть
2  CM_dH+_yellow_green_50%     CM_dH+_red_50%   1.738557e-77             1.043134e-76  Гипотеза отклонена. Различия есть
3          CM_dH+_blue_50%  CM_dH+_purple_50%  1.210989e-231            7.265931e-231  Гипотеза отклонена. Различия есть
4          CM_dH+_blue_50%     CM_dH+_red_50%  2.476211e-297            1.485727e-296  Гипотеза отклонена. Различия есть
5        CM_dH+_purple_50%     CM_dH+_red_50%   1.344621e-10             8.067725e-10  Гипотеза отклонена. Различия есть
